In [ ]:
import glob
import os
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from itertools import product

In [ ]:
np.random.seed(43)

In [ ]:
# Define paths to data
folder_paths = [
    '/home/sgurau/Desktop/output/functional_connectivity/AD/sub-*',
    '/home/sgurau/Desktop/output/functional_connectivity/CN/sub-*'
]

In [ ]:
# Create labels for data
labels_string = []
data_matrices = []  # List to store individual correlation matrices
for folder_path in folder_paths:
    if 'AD' in folder_path:
        label = 'AD'
    elif 'CN' in folder_path:
        label = 'CN'
    
    data_files = glob.glob(os.path.join(folder_path, '*.txt'))  
    for data_file in data_files:
        # Load correlation matrix from data_file
        correlation_matrix = np.loadtxt(data_file)
        data_matrices.append(correlation_matrix)
        labels_string.append(label)

In [ ]:
encoder = LabelEncoder()
labels = encoder.fit_transform(labels_string)

In [ ]:
# Load the geodesic distance matrix 
geodesic_distances_file = geodesic_distances_file = '/home/sgurau/Desktop/geodesic_distances.csv'
geodesic_distances = pd.read_csv(geodesic_distances_file, header=None).values

In [ ]:
# Define hyperparameter ranges
gamma_values = np.arange(10000, 1000000, 5000).tolist()
beta_values = np.arange(0.1, 2, 0.5).tolist()
C_values = np.arange(1, 100, 5).tolist()

In [ ]:

# Define the number of folds for cross-validation
kfold = 5

In [ ]:
# Stratification into folds
unique_labels, label_counts = np.unique(labels, return_counts=True)
folds = defaultdict(list)

In [ ]:
min_sizes = {label: count // kfold for label, count in zip(unique_labels, label_counts)}
remainders = {label: count % kfold for label, count in zip(unique_labels, label_counts)}

In [ ]:
for label in unique_labels:
    label_indices = np.where(labels == label)[0]
    np.random.shuffle(label_indices)
    current_fold = 0
    for index in label_indices:
        folds[current_fold].append(index)
        if len(folds[current_fold]) >= min_sizes[label] + (current_fold < remainders[label]):
            current_fold = (current_fold + 1) % kfold
           
for fold in folds:
    folds[fold] = np.array(folds[fold])

In [ ]:
# Create train and test indices for each fold
train_indices_all_folds = []
test_indices_all_folds = []

In [ ]:
for fold in range(kfold):
    test_indices = folds[fold]
    train_indices = np.hstack([folds[f] for f in range(kfold) if f != fold])
    train_indices_all_folds.append(train_indices)
    test_indices_all_folds.append(test_indices)

In [ ]:
# Perform cross-validation
accuracy_values = []

In [ ]:
# Initialize a dictionary to store the DataFrames for each fold
fold_results_dfs = {}

In [ ]:
for fold in range(kfold):
    train_indices = train_indices_all_folds[fold]
    test_indices = test_indices_all_folds[fold]
    for gamma in gamma_values:
            for beta in beta_values:
                precomputed_kernel_matrix = beta * np.exp(-geodesic_distances**2 / gamma)
        
                # Regularize the kernel matrix
                # precomputed_kernel_matrix += 5e-4 * np.eye(len(geodesic_distances))        
        
                for C in C_values:    
                    classifier = SVC(kernel='precomputed', C=C) # class_weight='balanced'
                    classifier.fit(precomputed_kernel_matrix[train_indices][:, train_indices], labels[train_indices])
                    predicted_labels = classifier.predict(precomputed_kernel_matrix[test_indices][:, train_indices])
                    accuracy = accuracy_score(labels[test_indices], predicted_labels)
                    accuracy_values.append(accuracy)
                    print(f"Fold={fold}, Gamma={gamma}, Beta={beta}, C={C}, Accuracy: {accuracy:.2f}")
            
                    # Create the DataFrame and store it in the fold_results_dfs dictionary
                    fold_results_dfs[fold] = pd.DataFrame({
                        'test_indices': test_indices,
                        'test_labels': labels[test_indices],
                        'predicted_labels': predicted_labels
                        })

In [ ]:
# Reshape the accuracy values for analysis
accuracy_values = np.array(accuracy_values).reshape(kfold, len(gamma_values), len(beta_values), -1)

In [ ]:
# Calculate the average accuracy and standard deviation across folds
average_accuracy = np.mean(accuracy_values, axis=0)
std_dev = np.std(accuracy_values, axis=0)

In [ ]:
# Combine gamma, beta, and C values for results
parameter_combinations = [(g, b, c) for g, b, c in product(gamma_values, beta_values, C_values)]
results_df = pd.DataFrame({
    'Gamma': [comb[0] for comb in parameter_combinations],
    'Beta': [comb[1] for comb in parameter_combinations],
    'C': [comb[2] for comb in parameter_combinations],
    'Average Accuracy': average_accuracy.flatten(),
    'Standard Deviation': std_dev.flatten()
})

In [ ]:
# Find the best combination of parameters
best_index = results_df['Average Accuracy'].idxmax()
best_parameters = results_df.loc[best_index]

In [ ]:
print("Best Parameters:")
print(best_parameters)

In [ ]:

# Calculate label distribution in the entire dataset
total_label_distribution = {label: np.sum(labels == label) for label in unique_labels}

In [ ]:
# Initialize a dictionary to hold the label distribution for each fold
fold_label_distributions = {fold: {label: 0 for label in unique_labels} for fold in range(kfold)}

In [ ]:
# Calculate label distribution for each fold
for fold in range(kfold):
    fold_labels = labels[test_indices_all_folds[fold]]
    for label in unique_labels:
        fold_label_distributions[fold][label] = np.sum(fold_labels == label)

In [ ]:
# Print distributions to compare them
print("Label distribution in the entire dataset:")
print(total_label_distribution)

In [ ]:
print("\nLabel distribution in each fold:")
for fold in range(kfold):
    print(f"Fold {fold}: {fold_label_distributions[fold]}")

In [ ]:
# Calculate the expected distribution in each fold (as a proportion)
expected_distribution = {label: count / len(labels) for label, count in total_label_distribution.items()}

In [ ]:
# Compare the fold distributions to the expected distribution
print("\nExpected label distribution (as proportions):")
print(expected_distribution)

In [ ]:
# Check if the folds are approximately equal to the expected distribution
for fold in range(kfold):
    fold_distribution_proportion = {
        label: fold_label_distributions[fold][label] / sum(fold_label_distributions[fold].values()) 
        for label in fold_label_distributions[fold]
    }
    print(f"Fold {fold} distribution as a proportion of total for each label:")
    print(fold_distribution_proportion)
    
# Dataframes to show train and test indices for each fold
# train_indices_df = pd.DataFrame(train_indices_all_folds, index=[f'Fold {i+1}' for i in range(kfold)])
# test_indices_df = pd.DataFrame(test_indices_all_folds, index=[f'Fold {i+1}' for i in range(kfold)])